In [33]:
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model
from energy_cycle import energy_generating_cycle_test

# read model for E. coli K12
k12 = read_sbml_model('../GEMs/iML1515.xml')

# CAL2 module

In [16]:
CAL2 = k12.copy()

## Model assembly
### Knock-outs

In [17]:
from cobra.manipulation import knock_out_model_genes

# pheA | gene b2599
knock_out_model_genes(CAL2, ["b2599"])

[<Reaction PPNDH at 0x12a6115a0>]

### Metabolites

In [18]:
#L-arogenate
Largn_c = Metabolite(
    'Largn_c',
    formula='C10H13NO5',
    name='L-Arogenate',
    compartment='c')

#p-Coumaric acid
coura_c = Metabolite(
    'coura_c',
    formula='C9H8O3',
    name='p-Coumaric acid',
    compartment='c')

#caffeic acid
caffa_c = Metabolite(
    'caffa_c',
    formula='C9H8O4',
    name='Caffeic acid',
    compartment='c')

#external caffeic acid
caffa_e = Metabolite(
    'caffa_e',
    formula='C9H8O4',
    name='Caffeic acid',
    compartment='e')

#ammonia
nh4_c = k12.metabolites.get_by_id("nh4_c")

#tyrosine 
tyr__L_c = k12.metabolites.get_by_id("tyr__L_c")

#Oxaloacetate
oaa_c = k12.metabolites.get_by_id("oaa_c")

#Aspartate
asp = k12.metabolites.get_by_id("asp__L_c")

#prephenate
pphn_c = k12.metabolites.get_by_id("pphn_c")

#NADH
nadh_c = k12.metabolites.get_by_id("nadh_c")

#NAD+
nad_c = k12.metabolites.get_by_id("nad_c")

#H+
h_c = k12.metabolites.get_by_id("h_c")

#ATP
atp_c = k12.metabolites.get_by_id("atp_c")

#AMP
adp_c = k12.metabolites.get_by_id("adp_c")

#Diphosphate
pi_c = k12.metabolites.get_by_id("pi_c")

### Reactions

In [20]:
#prep -> l-arogenate
PPNARO = Reaction('PPNARO')
PPNARO.name = 'L-arogenate:2-oxoglutarate aminotransferase'
PPNARO.subsystem = 'Phenylalanine, tyrosine and tryptophan biosynthesis'
PPNARO.lower_bound = -1000  # This is the default
PPNARO.upper_bound = 1000  # This is the default
#Oxaloacetate + L-Arogenate <=> L-Aspartate + Prephenate
PPNARO.add_metabolites({
    oaa_c: -1.0,
    Largn_c: -1.0,
    asp: 1.0,
    pphn_c: 1.0
})
PPNARO.gene_reaction_rule = 'b4054' #tyrB

CAL2.add_reactions([PPNARO])

In [21]:
#tyr -> p-coumaric acid
TAL = Reaction('TAL')
TAL.name = 'L-tyrosine ammonia-lyase'
TAL.subsystem = 'CA module'
TAL.lower_bound = -1000  # This is the default
TAL.upper_bound = 1000  # This is the default
#L-Tyrosine <=> 4-Coumarate + Ammonia
TAL.add_metabolites({
    tyr__L_c: -1.0,
    coura_c: 1.0,
    nh4_c: 1.0
})
#TAL.gene_reaction_rule = 'tal' #synthetic rgTAL gene

CAL2.add_reactions([TAL])

In [22]:
#p-coumaric acid -> caffeic acid
COURCA = Reaction('COURCA')
COURCA.name = '4-hydroxyphenylacetate 3-hydroxylase'
COURCA.subsystem = 'CA module'
COURCA.lower_bound = -1000  # This is the default
COURCA.upper_bound = 1000  # This is the default
#4-Coumarate <=> Caffeate
COURCA.add_metabolites({
    coura_c: -1.0,
    nadh_c: -1.0,
    caffa_c: 1.0,
    nad_c: 1.0,
    h_c: 1.0
})
#COURCA.gene_reaction_rule = '(hpaB and hpaC)' #synthetic hpaC gene

CAL2.add_reactions([COURCA])

In [23]:
#caffeic acid transport reaction
CAFFAt = Reaction('CAFFAt')
CAFFAt.name = 'caffa transport'
CAFFAt.subsystem = 'CA module'
CAFFAt.lower_bound = -1000  # This is the default
CAFFAt.upper_bound = 1000  # This is the default
CAFFAt.add_metabolites({
    caffa_c: -1.0,
    atp_c: -1.0,
    caffa_e: 1.0,
    adp_c: 1.0,
    pi_c: 1.0
})

CAL2.add_reactions([CAFFAt])

In [24]:
#add exhange reaction
CAL2.add_boundary(CAL2.metabolites.get_by_id("caffa_e"), type="exchange")

Reaction identifier,EX_caffa_e
Name,Caffeic acid exchange
Memory address,0x129dbb5b0
Stoichiometry,caffa_e <=> Caffeic acid <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Testing model

In [25]:
#test for energy-generating cycles
energy_generating_cycle_test(CAL2)

ATPM flux: 0.0


,fluxes,reduced_costs
CYTDK2,0.0,2.220446e-16
XPPT,0.0,9.871169e+00
HXPRT,0.0,9.871169e+00
NDPK5,0.0,-2.000000e+00
SHK3Dr,0.0,2.000000e+00
...,...,...
PPNARO,0.0,-2.000000e+00
TAL,0.0,-2.000000e+00
COURCA,0.0,-9.823077e-01
CAFFAt,0.0,2.000000e+00


In [26]:
#adding Phe to the medium
phe_medium = CAL2.medium
phe_medium["EX_phe__L_e"] = 1000.0
CAL2.medium = phe_medium

#stopping caffeic acid uptake so that the model does not use this as substrate
CAL2.reactions.EX_caffa_e.lower_bound = 0

In [27]:
from cobra.flux_analysis import flux_variability_analysis

CAL2.metabolites.caffa_c.summary(fva=0.95)

Percent,Flux,Range,Reaction,Definition
,0,[0; 0.2649],COURCA,coura_c + nadh_c <=> caffa_c + h_c + nad_c
Percent,Flux,Range,Reaction,Definition
,0,[-0.2649; 0],CAFFAt,atp_c + caffa_c <=> adp_c + caffa_e + pi_c


In [28]:
CAL2.metabolites.caffa_e.summary(fva=0.95)

Percent,Flux,Range,Reaction,Definition
,0,[0; 0.2649],CAFFAt,atp_c + caffa_c <=> adp_c + caffa_e + pi_c
Percent,Flux,Range,Reaction,Definition
,0,[-0.2649; 0],EX_caffa_e,caffa_e -->


## Export model to SBML

In [ ]:
from cobra.io import write_sbml_model

write_sbml_model(CAL2, "../GEMs/CAL2.xml")